In [ ]:
if (!requireNamespace("Seurat", quietly = TRUE)){
    install.packages("MLmetrics") 
    install.packages('Seurat')
    install.packages("remotes")
    remotes::install_github("mojaveazure/seurat-disk")
}

In [1]:
# run time record
time.start <- Sys.time()
print(paste('start time:',time.start))

[1] "start time: 2022-03-11 11:12:59"


In [2]:
library(MLmetrics)
library(Seurat)
library(SeuratDisk)
library(caret)


Attaching package: ‘MLmetrics’


The following object is masked from ‘package:base’:

    Recall


Attaching SeuratObject

Registered S3 method overwritten by 'cli':
  method     from         
  print.boxx spatstat.geom

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

Loading required package: ggplot2

Loading required package: lattice


Attaching package: ‘caret’


The following objects are masked from ‘package:MLmetrics’:

    MAE, RMSE




In [3]:
##h5ad data -> h5seurat data
#overwrite: if the file exist, then not  overwrite
if (!file.exists("../dataset/PBMC_CITE_modelC.h5seurat")){
    Convert ( "../dataset/PBMC_CITE_modelC.h5ad" , dest  =  "h5seurat" , overwrite  =  FALSE )
}
if (!file.exists("../dataset/PBMC68k_modelC.h5seurat")){
    Convert ( "../dataset/PBMC68k_modelC.h5ad" , dest  =  "h5seurat" , overwrite  =  FALSE )
}

In [4]:
ref_file <- "../dataset/PBMC_CITE_modelC.h5seurat"
test.file <- "../dataset/PBMC68k_modelC.h5seurat"

## Reference Data Preprocessing

In [5]:
ref <- LoadH5Seurat(ref_file)

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding feature-level metadata for RNA

Adding command information

Adding cell-level metadata

Warning message:
“Invalid name supplied, making object name syntactically valid. New object name is Phasecelltype.l1celltype.l2celltype.l3donorlanenCount_ADTnCount_RNAnCount_SCTnFeature_ADTnFeature_RNAnFeature_SCTorig.identtimemodelA.idmodelC.idtransfer.cell.typeorigin.cell.type; see ?make.names for more details on syntax validity”
Adding miscellaneous information

Adding tool-specific results



In [6]:
ref <- NormalizeData(ref, verbose = FALSE)
ref <- FindVariableFeatures(ref, selection.method = "vst", nfeatures = 2000,verbose = FALSE)

In [7]:
ref <- ScaleData(ref, verbose = FALSE)
ref <- RunPCA(ref, npcs = 30, verbose = FALSE)
ref <- RunUMAP(ref, reduction = "pca", dims = 1:30, verbose = FALSE)

Warning message:
“The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session”


## Cell type classification

In [8]:
test <- LoadH5Seurat(test.file)

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding feature-level metadata for RNA

Adding command information

Adding cell-level metadata

Warning message:
“Invalid name supplied, making object name syntactically valid. New object name is cell_ontology_classcell_ontology_idcell_type1dataset_namelatent_1latent_10latent_2latent_3latent_4latent_5latent_6latent_7latent_8latent_9organorganismplatformtSNE1tSNE2modelA.idmodelC.idtransfer.cell.typeorigin.cell.type; see ?make.names for more details on syntax validity”
Adding miscellaneous information

Adding tool-specific results



In [9]:
ref.anchors <- FindTransferAnchors(reference = ref, query =test, dims = 1:30, reference.reduction = "pca")

Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 5685 anchors

Filtering anchors

	Retained 214 anchors



In [10]:
ref$transfer_cell_type <- as.character(ref$modelC.id) # celltype = modelC.id (int to char)
predictions <- TransferData(anchorset = ref.anchors, refdata = ref$transfer_cell_type,dims = 1:30)
test <- AddMetaData(test, metadata = predictions)

Finding integration vectors

Finding integration vector weights

Predicting cell labels



## Make Confusion Matrix and evaluate accuracy

In [11]:
label <- c('T-helper cell','cytotoxic T cell','memory B cell','naive B cell',
           'plasma cell','natural killer cell','erythrocyte',
           'megakaryocyte','monocyte','dendritic cell','bone marrow hematopoietic cell')

In [12]:
y.pred <- factor(label[as.integer(test$predicted.id)+1],levels=label)
y.true <- factor(test$transfer.cell.type,levels=label)
y.pred <- y.pred[!is.na(y.true)]
y.true <- y.true[!is.na(y.true)]

In [13]:
mat.table <- confusionMatrix(y.true,y.pred)
mat.df <- as.data.frame.matrix(mat.table$table) 
colnames(mat.df) = 0:10
mat.df

,0,1,2,3,4,5,6,7,8,9,10
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
T-helper cell,41934,0,0,0,9,0,0,0,173,16,47
cytotoxic T cell,22139,0,0,0,1,0,0,0,19,0,3
memory B cell,0,0,0,0,0,0,0,0,0,0,0
naive B cell,8929,0,0,0,1,0,0,0,1005,150,0
plasma cell,0,0,0,0,0,0,0,0,0,0,0
natural killer cell,7661,0,0,0,0,0,0,0,704,0,20
erythrocyte,0,0,0,0,0,0,0,0,0,0,0
megakaryocyte,0,0,0,0,0,0,0,0,0,0,0
monocyte,44,0,0,0,1,0,0,0,2378,44,0


In [14]:
mat.df.percent <- round(mat.df/(rowSums(mat.df)+0.001)*100, digits=1)
mat.df.percent
print(paste('acc:',sum(y.true==y.pred)/length(y.true)*100,'%'))

,0,1,2,3,4,5,6,7,8,9,10
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
T-helper cell,99.4,0,0,0,0,0,0,0,0.4,0.0,0.1
cytotoxic T cell,99.9,0,0,0,0,0,0,0,0.1,0.0,0.0
memory B cell,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0
naive B cell,88.5,0,0,0,0,0,0,0,10.0,1.5,0.0
plasma cell,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0
natural killer cell,91.4,0,0,0,0,0,0,0,8.4,0.0,0.2
erythrocyte,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0
megakaryocyte,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0
monocyte,1.8,0,0,0,0,0,0,0,96.4,1.8,0.0


[1] "acc: 52.2426434540087 %"


In [15]:
evaluation_metrics<-function(performance){
    eva.list <- c()
    performance.name <- as.character(substitute(performance))
    for (i in label)
    {
        eva <- performance(y.true,y.pred, positive = i)
        if (!is.nan(eva))
        {
            eva.list <- c(eva.list,eva)
        }
    }
    print(paste(c(paste(performance.name,':'),eva.list)))
    print('-----------------------------------------------------------------')
    print(paste(performance.name,'(average):',mean(eva.list)))
}

In [16]:
evaluation_metrics(Precision)

[1] "Precision :"       "0.508759584587014" "0"                
[4] "0.44020733061829"  "0.275757575757576" "0.980662983425414"
[1] "-----------------------------------------------------------------"
[1] "Precision (average): 0.441077494877659"


In [17]:
evaluation_metrics(Recall)

[1] "Recall :"          "0.994191422271747" "0"                
[4] "0"                 "0"                 "0.963923794081881"
[7] "0.919191919191919" "0.553649407361198"
[1] "-----------------------------------------------------------------"
[1] "Recall (average): 0.490136648986678"


In [18]:
evaluation_metrics(F1_Score)

[1] "F1_Score :"        "0.67308170750303"  "0.604397000889567"
[4] "0.424242424242424" "0.707735247208931"
[1] "-----------------------------------------------------------------"
[1] "F1_Score (average): 0.602364094960988"


In [19]:
# run time record
time.end <- Sys.time()
print(paste('start time:',time.start))
print(paste('end   time:',time.end))
time.end - time.start 

[1] "start time: 2022-03-11 11:12:59"
[1] "end   time: 2022-03-11 11:56:16"


Time difference of 43.29046 mins

In [20]:
# !rm ../dataset/PBMC_CITE_modelC.h5ad
# !rm ../dataset/CordBlood_modelC.h5ad